In [72]:
import pandas as pd
import numpy as np
import ast

In [ ]:
interval_df = pd.read_csv("~/Downloads/sherlock/valid_7day_windows.csv")
interval_df.head()

,healthCode,time_range,file_uris
0,1b75ede0-69d3-41f1-9756-4025e4d19011,2019-03-09_2019-03-15,['1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03...
1,1b75ede0-69d3-41f1-9756-4025e4d19011,2019-03-16_2019-03-22,['1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03...
2,1b75ede0-69d3-41f1-9756-4025e4d19011,2019-03-23_2019-03-29,['1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03...
3,1b75ede0-69d3-41f1-9756-4025e4d19011,2019-04-05_2019-04-11,['1b75ede0-69d3-41f1-9756-4025e4d19011/2019-04...
4,1b75ede0-69d3-41f1-9756-4025e4d19011,2019-05-12_2019-05-18,['1b75ede0-69d3-41f1-9756-4025e4d19011/2019-05...


In [75]:
import pandas as pd
import numpy as np
import ast
import re # Import regular expressions module


# Updated function to handle standard lists and space-separated number lists
def parse_string_list_safely(item):
    if pd.isna(item):
        return np.nan
    elif isinstance(item, list):
         return item # Already a list
    elif isinstance(item, str):
        original_item_repr = repr(item) # Get a representation for logging
        item = item.strip() # Remove leading/trailing whitespace
        if not item:
            return np.nan # Handle empty strings

        # Attempt 1: Standard Python list literal parsing
        try:
            evaluated = ast.literal_eval(item)
            if isinstance(evaluated, list):
                return evaluated
            else:
                # If it evaluates to a single item, wrap it in a list?
                # Example: "'[10.]'" might eval to 10.0 if quotes were weird.
                # Let's assume we always want list output from strings like '[...]'
                # If the original string looks like a list, but evaluates to single item:
                if item.startswith('[') and item.endswith(']'):
                     return [evaluated] # Wrap single valid item in list
                else:
                     # Or treat non-list results from valid literals as error?
                     print(f"Warning: Evaluated item {original_item_repr} is not a list, treating as NaN.")
                     return np.nan
        except (ValueError, SyntaxError, TypeError):
            # If standard parsing fails, proceed to Attempt 2
            pass # Fall through to the next attempt

        # Attempt 2: Space-separated numbers within brackets
        if item.startswith('[') and item.endswith(']'):
            content = item[1:-1].strip() # Extract content within brackets
            if not content: # Handle empty brackets '[]'
                return []
            try:
                # Split by whitespace, filter empty strings, convert to float
                # Using regex \s+ handles multiple spaces correctly
                parsed_list = [float(num_str) for num_str in re.split(r'\s+', content) if num_str]
                return parsed_list
            except ValueError:
                # Handle cases where content isn't purely space-separated numbers
                print(f"Warning: Could not parse content of {original_item_repr} as space-separated numbers, treating as NaN.")
                return np.nan
            except Exception as e:
                 print(f"Warning: Unexpected error parsing space-separated list {original_item_repr}: {e}, treating as NaN.")
                 return np.nan

        # If neither parsing method worked
        print(f"Warning: Could not parse string {original_item_repr} as any known list format, treating as NaN.")
        return np.nan
    else:
        print(f"Warning: Unexpected type {type(item)} for item '{item}', treating as NaN.")
        return np.nan

In [ ]:
label_df = pd.read_csv("~/Downloads/sherlock/combined_mhc_data.csv")
label_df.createdOn = pd.to_datetime(label_df.createdOn, format='ISO8601')
label_df.heart_disease = label_df.heart_disease.apply(parse_string_list_safely)
label_df = label_df.explode('heart_disease')
label_df.head()

,healthCode,createdOn,sleep_diagnosis1,sleep_diagnosis2,happiness,heart_disease,feel_worthwhile1,feel_worthwhile2,feel_worthwhile3,Diabetes,Hypertension,BloodGlucose
0,-2Zw_7F7KSEkrt5aUBQCwOtW,2020-03-19 21:32:21.021,False,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-2Zw_7F7KSEkrt5aUBQCwOtW,2020-03-20 06:34:08.026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-2Zw_7F7KSEkrt5aUBQCwOtW,2020-03-20 06:39:53.985,NaN,NaN,NaN,NaN,7.0,8.0,2.0,NaN,NaN,NaN
3,-2Zw_7F7KSEkrt5aUBQCwOtW,2020-03-20 06:40:34.719,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN
4,-2Zw_7F7KSEkrt5aUBQCwOtW,2020-03-21 08:03:20.856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
labels = ['sleep_diagnosis1', 'happiness', 'heart_disease', 'feel_worthwhile1', 
          'feel_worthwhile2', 'feel_worthwhile3', 'Diabetes', 'Hypertension']

for label in labels:
    label_df[label] = label_df[label].apply(lambda x: np.nan if np.isnan(x) else int(x))

In [121]:
hcs = list(interval_df.healthCode.unique())

global_records = []
for hc in hcs:
    i_df = interval_df[interval_df.healthCode == hc].copy()
    if i_df.empty:
        continue
    
    records = i_df.to_dict('records')

    for label in labels:
        l_df = label_df[(label_df.healthCode == hc) & (label_df[label].notna())].copy()
        if l_df.empty:
            continue
        dates = l_df.createdOn.dt.date.astype(str)
        matches = find_closest_dates(dates, i_df.time_range)
        for record, match_key in zip(records, matches):
            match_idx = matches[match_key]
            label_dict = {
                'label_value': l_df.iloc[match_idx][label],
                'label_date': l_df.iloc[match_idx]['createdOn']
            }
            record[label] = label_dict
    global_records.extend(records)

In [123]:
import json
import os
from pathlib import Path

# Convert numpy values to Python native types for JSON serialization
def convert_numpy_to_native(obj):
    if isinstance(obj, dict):
        return {k: convert_numpy_to_native(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy_to_native(item) for item in obj]
    elif hasattr(obj, 'item'):  # Check if it's a numpy scalar
        return obj.item()
    else:
        return obj

# Convert records to JSON-serializable format
serializable_records = convert_numpy_to_native(global_records)

# Create output path
output_path = os.path.expanduser("~/Downloads/global_records.json")

# Write to JSON file
with open(output_path, 'w') as f:
    json.dump(serializable_records, f, indent=2, default=str)

# Display first record as confirmation
global_records[0] if global_records else "No records found"

{'healthCode': '1b75ede0-69d3-41f1-9756-4025e4d19011',
 'time_range': '2019-03-09_2019-03-15',
 'file_uris': "['1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03-11.npy', '1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03-12.npy', '1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03-13.npy', '1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03-14.npy', '1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03-15.npy']",
 'happiness': {'label_value': np.float64(10.0),
  'label_date': Timestamp('2019-03-21 13:09:23')}}

In [124]:
# Create a denormalized pandas dataframe from global_records
import pandas as pd

# First, flatten nested dictionaries in global_records
flattened_records = []
for record in global_records:
    flat_record = {}
    for key, value in record.items():
        if isinstance(value, dict) and 'label_value' in value:
            # Flatten label dictionaries
            flat_record[f"{key}_value"] = value['label_value']
            flat_record[f"{key}_date"] = value['label_date']
        else:
            flat_record[key] = value
    flattened_records.append(flat_record)

# Create pandas DataFrame from flattened records
denormalized_df = pd.DataFrame(flattened_records)

# Display the first few rows
denormalized_df.head()

,healthCode,time_range,file_uris,happiness_value,happiness_date,sleep_diagnosis1_value,sleep_diagnosis1_date,feel_worthwhile1_value,feel_worthwhile1_date,feel_worthwhile2_value,feel_worthwhile2_date,heart_disease_value,heart_disease_date,feel_worthwhile3_value,feel_worthwhile3_date,Diabetes_value,Diabetes_date,Hypertension_value,Hypertension_date
0,1b75ede0-69d3-41f1-9756-4025e4d19011,2019-03-09_2019-03-15,['1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03...,10.0,2019-03-21 13:09:23,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
1,1b75ede0-69d3-41f1-9756-4025e4d19011,2019-03-16_2019-03-22,['1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03...,10.0,2019-03-21 13:09:23,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
2,1b75ede0-69d3-41f1-9756-4025e4d19011,2019-03-23_2019-03-29,['1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03...,10.0,2019-03-23 13:00:43,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
3,1b75ede0-69d3-41f1-9756-4025e4d19011,2019-04-05_2019-04-11,['1b75ede0-69d3-41f1-9756-4025e4d19011/2019-04...,10.0,2019-03-23 13:00:43,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
4,1b75ede0-69d3-41f1-9756-4025e4d19011,2019-05-12_2019-05-18,['1b75ede0-69d3-41f1-9756-4025e4d19011/2019-05...,10.0,2019-03-23 13:00:43,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT


In [126]:
denormalized_df.to_parquet("~/Downloads/global_records.parquet")

In [87]:
# Display value counts for each column in label_df
for column in labels:
    if column in ['healthCode', 'createdOn', 'file_uris']:
        # Skip non-categorical columns
        continue
    print(label_df[column].value_counts())

sleep_diagnosis1
0.0    48978
1.0     7563
Name: count, dtype: int64
happiness
8.0     10759
9.0      8550
7.0      6804
10.0     5908
6.0      3597
5.0      2749
4.0      1399
3.0       923
2.0       541
0.0       342
1.0       267
Name: count, dtype: int64
heart_disease
10.0    34376
8.0      1649
4.0      1146
1.0      1059
5.0      1007
9.0       944
3.0       866
2.0       608
7.0       459
6.0       299
11.0      227
Name: count, dtype: int64
feel_worthwhile1
8.0     15508
9.0     12399
7.0      9946
10.0     9110
6.0      5047
5.0      4592
4.0      2194
3.0      1584
2.0       999
1.0       531
0.0       278
Name: count, dtype: int64
feel_worthwhile2
8.0     15241
9.0     11507
7.0     11300
10.0     6343
6.0      5974
5.0      5047
4.0      2687
3.0      1831
2.0      1227
1.0       710
0.0       340
Name: count, dtype: int64
feel_worthwhile3
2.0     9143
3.0     7931
1.0     7169
5.0     6764
7.0     5896
8.0     5642
4.0     5553
6.0     5037
0.0     4260
9.0     2793
10.0  

In [18]:
from datetime import datetime
import bisect

def find_closest_dates(dates_list, intervals_list):
    """
    For each interval, find the closest date in the dates_list.
    
    Args:
        dates_list: List of date strings in format 'YYYY-MM-DD'
        intervals_list: List of interval strings in format 'YYYY-MM-DD_YYYY-MM-DD'
        
    Returns:
        Dictionary with intervals as keys and indices of closest dates as values
    """
    # Convert dates_list to datetime objects
    dates_dt = [datetime.strptime(date, "%Y-%m-%d") for date in dates_list]
    
    # Sort dates for binary search
    sorted_dates_with_indices = sorted(enumerate(dates_dt), key=lambda x: x[1])
    sorted_dates = [d for _, d in sorted_dates_with_indices]
    original_indices = [i for i, _ in sorted_dates_with_indices]
    
    result = {}
    
    for interval in intervals_list:
        # Parse start and end dates
        start_str, end_str = interval.split('_')
        start_date = datetime.strptime(start_str, "%Y-%m-%d")
        end_date = datetime.strptime(end_str, "%Y-%m-%d")
        
        # Calculate midpoint
        midpoint = start_date + (end_date - start_date) / 2
        
        # Find the closest date using binary search
        pos = bisect.bisect_left(sorted_dates, midpoint)
        
        # Handle edge cases
        if pos == 0:
            closest_idx = 0
        elif pos == len(sorted_dates):
            closest_idx = len(sorted_dates) - 1
        else:
            # Check which is closer
            if abs(sorted_dates[pos] - midpoint) < abs(sorted_dates[pos-1] - midpoint):
                closest_idx = pos
            else:
                closest_idx = pos - 1
        
        # Get original index
        original_idx = original_indices[closest_idx]
        result[interval] = original_idx
    
    return result

In [144]:
df0 = pd.read_parquet("~/Downloads/global_records.parquet")
df1 = pd.read_parquet("~/Downloads/global_records_1.parquet")

In [145]:
# Compare the two dataframes
print("Shape comparison:")
print(f"df0 shape: {df0.shape}")
print(f"df1 shape: {df1.shape}")

# Check if they have the same columns
print("\nColumn comparison:")
df0_cols = set(df0.columns)
df1_cols = set(df1.columns)
print(f"Columns only in df0: {df0_cols - df1_cols}")
print(f"Columns only in df1: {df1_cols - df0_cols}")
print(f"Common columns: {len(df0_cols.intersection(df1_cols))}")

# Compare values for common columns
common_cols = list(df0_cols.intersection(df1_cols))
print("\nValue comparison for common columns:")
for col in common_cols[:5]:  # Limit to first 5 columns to avoid excessive output
    if df0[col].equals(df1[col]):
        print(f"Column '{col}' has identical values")
    else:
        print(f"Column '{col}' has differences")
        # Show a sample of differences
        mask = ~(df0[col] == df1[col])
        if mask.any():
            diff_sample = pd.DataFrame({
                'df0': df0.loc[mask, col].head(3),
                'df1': df1.loc[mask, col].head(3)
            })
            print(diff_sample)

# Check for NaN differences
print("\nNaN value comparison:")
nan_diff_cols = []
for col in common_cols:
    df0_nulls = df0[col].isna().sum()
    df1_nulls = df1[col].isna().sum()
    if df0_nulls != df1_nulls:
        nan_diff_cols.append((col, df0_nulls, df1_nulls))

if nan_diff_cols:
    print("Columns with different NaN counts:")
    for col, df0_nulls, df1_nulls in nan_diff_cols[:5]:  # Limit to first 5
        print(f"'{col}': df0={df0_nulls} nulls, df1={df1_nulls} nulls")
else:
    print("All columns have the same number of NaN values")

# Check for duplicate rows
print("\nDuplicate row comparison:")
print(f"df0 duplicated rows: {df0.duplicated().sum()}")
print(f"df1 duplicated rows: {df1.duplicated().sum()}")

# Summary statistics comparison for numeric columns
print("\nSummary statistics comparison (first numeric column):")
numeric_cols = [col for col in common_cols if pd.api.types.is_numeric_dtype(df0[col])]
if numeric_cols:
    col = numeric_cols[0]
    print(f"Column: {col}")
    print("df0 stats:")
    print(df0[col].describe())
    print("df1 stats:")
    print(df1[col].describe())


Shape comparison:
df0 shape: (600706, 19)
df1 shape: (600706, 19)

Column comparison:
Columns only in df0: set()
Columns only in df1: set()
Common columns: 19

Value comparison for common columns:
Column 'feel_worthwhile2_date' has identical values
Column 'feel_worthwhile3_date' has identical values
Column 'sleep_diagnosis1_date' has identical values
Column 'Diabetes_date' has identical values
Column 'Hypertension_date' has identical values

NaN value comparison:
All columns have the same number of NaN values

Duplicate row comparison:
df0 duplicated rows: 0
df1 duplicated rows: 0

Summary statistics comparison (first numeric column):
Column: Diabetes_value
df0 stats:
count    256217.000000
mean          0.067958
std           0.251674
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: Diabetes_value, dtype: float64
df1 stats:
count    256217.000000
mean          0.067958
std           0.251674
min           0.000000

In [141]:
# Load the global_records1.json file
import json

with open('/Users/narayanschuetz/Downloads/global_records_1.json', 'r') as f:
    global_records = json.load(f)
    
# Display the loaded data
global_records

[{'healthCode': '1b75ede0-69d3-41f1-9756-4025e4d19011',
  'time_range': '2019-03-09_2019-03-15',
  'file_uris': "['1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03-11.npy', '1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03-12.npy', '1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03-13.npy', '1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03-14.npy', '1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03-15.npy']",
  'happiness': {'label_value': 10.0, 'label_date': '2019-03-21 13:09:23'}},
 {'healthCode': '1b75ede0-69d3-41f1-9756-4025e4d19011',
  'time_range': '2019-03-16_2019-03-22',
  'file_uris': "['1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03-16.npy', '1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03-17.npy', '1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03-18.npy', '1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03-19.npy', '1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03-20.npy', '1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03-21.npy']",
  'happiness': {'label_value': 10.0, 'label_date': '2019-03-21 13:09:23'}},
 {'healthCo

In [142]:
# Check if 'sleep_diagnosis1' key exists in any dictionary in global_records
sleep_diagnosis1_exists = any('sleep_diagnosis1' in record for record in global_records)
print(f"Does 'sleep_diagnosis1' key exist in any record? {sleep_diagnosis1_exists}")

# Display records that have the 'sleep_diagnosis1' key (if any)
records_with_sleep_diagnosis1 = [record for record in global_records if 'sleep_diagnosis1' in record]
if records_with_sleep_diagnosis1:
    print(f"\nFound {len(records_with_sleep_diagnosis1)} records with 'sleep_diagnosis1' key:")
    for i, record in enumerate(records_with_sleep_diagnosis1[:3]):  # Show first 3 examples
        print(f"Record {i+1}: {record}")
    if len(records_with_sleep_diagnosis1) > 3:
        print(f"... and {len(records_with_sleep_diagnosis1) - 3} more records")
else:
    print("No records contain the 'sleep_diagnosis1' key")

# Still display the global_records for reference
global_records

Does 'sleep_diagnosis1' key exist in any record? True

Found 482193 records with 'sleep_diagnosis1' key:
Record 1: {'healthCode': '1e536336-c4ff-43f1-a6cd-b6873ae171ee', 'time_range': '2016-07-20_2016-07-26', 'file_uris': "['1e536336-c4ff-43f1-a6cd-b6873ae171ee/2016-07-20.npy', '1e536336-c4ff-43f1-a6cd-b6873ae171ee/2016-07-21.npy', '1e536336-c4ff-43f1-a6cd-b6873ae171ee/2016-07-22.npy', '1e536336-c4ff-43f1-a6cd-b6873ae171ee/2016-07-23.npy', '1e536336-c4ff-43f1-a6cd-b6873ae171ee/2016-07-24.npy', '1e536336-c4ff-43f1-a6cd-b6873ae171ee/2016-07-25.npy', '1e536336-c4ff-43f1-a6cd-b6873ae171ee/2016-07-26.npy']", 'sleep_diagnosis1': {'label_value': 0.0, 'label_date': '2015-03-10 04:58:23'}, 'feel_worthwhile1': {'label_value': 7.0, 'label_date': '2015-03-11 02:57:10'}, 'feel_worthwhile2': {'label_value': 5.0, 'label_date': '2015-03-11 02:57:10'}}
Record 2: {'healthCode': '1e536336-c4ff-43f1-a6cd-b6873ae171ee', 'time_range': '2016-07-27_2016-08-02', 'file_uris': "['1e536336-c4ff-43f1-a6cd-b6873ae1

[{'healthCode': '1b75ede0-69d3-41f1-9756-4025e4d19011',
  'time_range': '2019-03-09_2019-03-15',
  'file_uris': "['1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03-11.npy', '1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03-12.npy', '1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03-13.npy', '1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03-14.npy', '1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03-15.npy']",
  'happiness': {'label_value': 10.0, 'label_date': '2019-03-21 13:09:23'}},
 {'healthCode': '1b75ede0-69d3-41f1-9756-4025e4d19011',
  'time_range': '2019-03-16_2019-03-22',
  'file_uris': "['1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03-16.npy', '1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03-17.npy', '1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03-18.npy', '1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03-19.npy', '1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03-20.npy', '1b75ede0-69d3-41f1-9756-4025e4d19011/2019-03-21.npy']",
  'happiness': {'label_value': 10.0, 'label_date': '2019-03-21 13:09:23'}},
 {'healthCo

In [147]:
# Find healthCodes where there are no labels at all
# First identify all label columns (those ending with '_value')
label_columns = [col for col in df1.columns if col.endswith('_value')]

# For each healthCode, check if all label columns are NaN
mask = df1[label_columns].isna().all(axis=1)
healthcodes_without_labels = df1.loc[mask, 'healthCode'].unique()

print(f"Found {len(healthcodes_without_labels)} healthCodes with no labels:")
print(healthcodes_without_labels[:10])  # Show first 10 as example
if len(healthcodes_without_labels) > 10:
    print(f"... and {len(healthcodes_without_labels) - 10} more")


Found 934 healthCodes with no labels:
['ca6c4504-cd3f-432b-98b0-bcde8aa37943' 'B05yq0JJPcXRljkEy50Cs5IZ'
 'f9ad77dc-0646-4639-8b31-b652c4285c44' 'Z1SZQARYlxSBKNTgmIrV8FLq'
 'ZgaUYoMOsXifURNuI0W6qQW3' 'UcDEuU0cUn987z8ib_JUIZjR'
 '4efb8ae1-4323-4874-af4b-ee5226df756d' 'mUBekI6w_eYCNbNM4JM6YP_K'
 'f30a3ac9-40bc-4777-a7cd-63231f83703b' 'FX4wj-WsxBd_h-vng6LYNFoI']
... and 924 more
